In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
import time
import pandas as pd

from TradingGym.OrderFlow import OrderFlow, readTxt
from TradingGym.OrderBook import OrderBook

In [ ]:
hdf_path = '../../Data/Si-3.18/hdf5/Si-3_18.h5'

In [ ]:
keys = []
with pd.HDFStore(hdf_path) as store:
    keys = store.keys()
print(keys)

In [ ]:
flow = OrderFlow()
flow.df = pd.read_hdf(hdf_path, key=keys[0])

In [ ]:
book = OrderBook()

In [ ]:
max_length = 10**5
right_before_trading = flow.df[flow.df.Flags.str.contains('Snapshot')].iloc[-1]
trading_start = flow.df.iloc[right_before_trading.name + 1]
trading_end = flow.df.iloc[min(max_length + trading_start.name, len(flow.df)) - 1]
total_time = trading_end.ExchTime.value - trading_start.ExchTime.value
start = trading_start.name
end = trading_end.name
step = 0.1 # in seconds
width = 100 # bp around mid price to show

In [ ]:
def BookIter(end):
    for idx in range(end):
        book.update(flow.df.iloc[idx])
        yield book, flow.df.iloc[idx]

In [ ]:
bookIter = BookIter(end)

In [ ]:
for idx in range(start):
    next(bookIter)

In [ ]:
plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
fig.show()
fig.canvas.draw()

In [ ]:
tooSlow = False
for book, message in bookIter:
    if not 'EndOfTransaction' in message.Flags:
        continue
        
    stTime = time.process_time()
    
    midPrice = (max(book.book[0].keys()) + min(book.book[1].keys())) / 2

    ax.clear()
    ax.set_xlim((midPrice - width / 2, midPrice + width / 2))
    ax.set_ylim((0, 8))

    bar1 = ax.bar([x for x in book.book[0].keys()], [math.log(x) for x in book.book[0].values()], color='r')
    bar2 = ax.bar([x for x in book.book[1].keys()], [math.log(x) for x in book.book[1].values()], color='b')
    ax.set_title(str(message.ExchTime))
    
    fig.canvas.draw()
    enTime = time.process_time()
    remaining = step - enTime + stTime
    if (remaining < 0 and not tooSlow):
        print("Warning: the plot is too slow: {} vs. {:.4f}".format(step, enTime - stTime))
        tooSlow = True
    time.sleep(max(0, remaining))

In [ ]:
ts = []
prices = []
book = OrderBook()

percentage_done = 0.0

print('Started simulation from time: {}'.format(trading_start.ExchTime))
print('Planned end time: {}'.format(trading_end.ExchTime))
        
for name, deal in flow.df.iterrows():
    if name > end:
        break
    book.update(deal)
    if (name >= start and 'EndOfTransaction' in deal.Flags):
        ts.append(deal.ExchTime)
        
        midPrice = (max(book.book[0].keys()) + min(book.book[1].keys())) / 2
        prices.append(midPrice)
    
    new_percentage_done = 100.0 - (trading_end.ExchTime.value - deal.ExchTime.value) / total_time * 100
    if (new_percentage_done > percentage_done + 1.0):
        percentage_done = new_percentage_done
        print('Done {:.0f}%'.format(percentage_done))

In [ ]:
def plot(panel, title, col, inter):
    assert(len(panel) == len(col))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    for idx in range(len(panel)):
        ax.plot(panel[idx][0], panel[idx][1], col[idx])

    hours = mdates.HourLocator()   # every hour
    hoursFmt = mdates.DateFormatter('')
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hoursFmt)

    minutes = mdates.MinuteLocator(interval=inter)  # every 10 minutes
    minutesFmt = mdates.DateFormatter('%D %H:%M')
    ax.xaxis.set_minor_locator(minutes)
    ax.xaxis.set_minor_formatter(minutesFmt)

    plt.setp(ax.xaxis.get_minorticklabels(), size=8, rotation=20)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)

    fig.tight_layout()
    fig.show()

In [ ]:
plot([[ts, prices]], 'Prices', ['r'], 10)